## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

# Подготовка данных

In [1]:
import pandas as pd 
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, Imputer, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator

In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

In [3]:
df_housing = spark.read.load('/datasets/housing.csv', format='csv',\
                             inferSchema=True, header='true')
df_housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [4]:
df_housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


In [5]:
df_housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [6]:
columns = df_housing.columns

for column in columns:
    check_col = F.col(column).cast('float').isNull()
    print(column, df_housing.filter(check_col).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 20640


In [7]:
all_cols = df_housing.columns
all_cols

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']

In [8]:
cats = ['ocean_proximity']
nums = ['longitude', 'latitude', 'housing_median_age',\
        'total_rooms', 'total_bedrooms', 'population',\
        'households', 'median_income']
target = 'median_house_value'

In [9]:
imputer = Imputer(inputCols=['total_bedrooms'],
                  outputCols=['total_bedrooms']).setStrategy("median")

df_housing = imputer.fit(df_housing).transform(df_housing)

df_housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,536.8388565891473,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.39187792168866,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


### Разделение на выборки

In [10]:
train_data, test_data = df_housing.randomSplit([.8,.2], seed=42)
print(train_data.count(), test_data.count())

16560 4080


### Трансформация категорийных признаков

In [11]:
df_housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [12]:
indexer = StringIndexer(inputCols=cats, 
                        outputCols=[c+'_idx' for c in cats])\
                       .fit(train_data)

train_data = indexer.transform(train_data)
test_data = indexer.transform(test_data)

In [13]:
df_housing.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']

In [14]:
cols = [c for c in df_housing.columns for i in cats if (c.startswith(i))]
df_housing.select(cols).show(5) 

+---------------+
|ocean_proximity|
+---------------+
|       NEAR BAY|
|       NEAR BAY|
|       NEAR BAY|
|       NEAR BAY|
|       NEAR BAY|
+---------------+
only showing top 5 rows



In [15]:
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in cats],
                        outputCols=[c+'_ohe' for c in cats])\
                       .fit(train_data)

train_data = encoder.transform(train_data)
test_data = encoder.transform(test_data)

In [16]:
cols = [c for c in train_data.columns for i in cats if (c.startswith(i))]
train_data.select(cols).show(5) 

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
+---------------+-------------------+-------------------+
only showing top 5 rows



In [17]:
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in cats],
                                        outputCol="categorical_features")

train_data = categorical_assembler.transform(train_data) 
test_data = categorical_assembler.transform(test_data) 

### Трансформация числовых признаков

In [18]:
numerical_assembler = VectorAssembler(inputCols=nums, 
                                      outputCol="numerical_features")

train_data = numerical_assembler.transform(train_data)
test_data = numerical_assembler.transform(test_data)

In [19]:
standardScaler = StandardScaler(inputCol='numerical_features',
                                outputCol="numerical_features_scaled")\
                               .fit(train_data) 

train_data = standardScaler.transform(train_data) 
test_data = standardScaler.transform(test_data) 

In [20]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 

train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)

train_data.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-61.931952286653...|
|       (4,[2],[1.0])|     [-61.907050013920...|
|       (4,[2],[1.0])|     [-61.892108650280...|
+--------------------+-------------------------+
only showing top 3 rows



# Обучение моделей

## Linear с полным набором параметров

In [21]:
lr = LinearRegression(regParam=0.001, labelCol=target, featuresCol='features')

model = lr.fit(train_data)

23/02/21 06:21:34 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/02/21 06:21:34 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/02/21 06:21:34 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/02/21 06:21:34 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [22]:
predictions = model.transform(test_data)

predictedLabes = predictions.select(target, "prediction")
predictedLabes.show() 

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|150533.05201137392|
|          106700.0|217624.65557502257|
|           73200.0|125169.93731395248|
|           90100.0|195019.31856626645|
|           67000.0|152291.14378080424|
|           86400.0| 186076.1646265802|
|           70500.0|  163936.411388007|
|           85100.0|180008.60364678735|
|           80500.0|181580.62083134847|
|           96000.0|170314.91081720544|
|           75500.0|137047.88310815254|
|           75000.0|104686.28871561633|
|          100600.0|191108.09587061638|
|           74100.0|156926.54312393768|
|           66800.0|134358.35757881217|
|           72600.0|161405.11234061373|
|           81100.0|150431.96760939434|
|          135600.0|174070.42442664923|
|          119400.0|166828.34705348825|
|           71300.0|169746.66898164805|
+------------------+------------------+
only showing top 20 rows



### Оценка модели

In [23]:
rsme = RegressionEvaluator(labelCol=target,\
                           metricName='rmse').evaluate(predictions)

mae = RegressionEvaluator(labelCol=target,\
                          metricName='mae').evaluate(predictions)

r2 = RegressionEvaluator(labelCol=target,\
                         metricName='r2').evaluate(predictions)

print('RSME: ', rsme)
print('MAE: ', mae) 
print('R2: ', r2)

RSME:  70786.46256245082
MAE:  50863.55254301572
R2:  0.6378452502238174


## Linear без категориальных переменных

In [24]:
lr = LinearRegression(regParam=0.001, labelCol=target, featuresCol='numerical_features_scaled')

model = lr.fit(train_data)

In [25]:
predictions = model.transform(test_data)

predictedLabes = predictions.select(target, "prediction")
predictedLabes.show() 

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|100751.20763066737|
|          106700.0|190787.24162771692|
|           73200.0| 74736.22486875672|
|           90100.0|162342.37158293743|
|           67000.0| 119469.7392479456|
|           86400.0|155921.97273852723|
|           70500.0|131193.75734864315|
|           85100.0| 150446.1475033695|
|           80500.0|150167.76686676592|
|           96000.0|133759.40356000653|
|           75500.0| 99083.86418192228|
|           75000.0|50512.753887748346|
|          100600.0|157630.49641811615|
|           74100.0|123925.08694595052|
|           66800.0|103050.25701026153|
|           72600.0|127714.79813263332|
|           81100.0|116199.38302141055|
|          135600.0| 150752.7982377666|
|          119400.0|126716.72205311386|
|           71300.0|145282.65911599854|
+------------------+------------------+
only showing top 20 rows



### Оценка модели

In [26]:
rsme = RegressionEvaluator(labelCol=target,\
                           metricName='rmse').evaluate(predictions)

mae = RegressionEvaluator(labelCol=target,\
                          metricName='mae').evaluate(predictions)

r2 = RegressionEvaluator(labelCol=target,\
                         metricName='r2').evaluate(predictions)

print('RSME: ', rsme)
print('MAE: ', mae) 
print('R2: ', r2)

RSME:  71791.28367374057
MAE:  51804.392993042435
R2:  0.6274906279118236


In [27]:
spark.stop()

# Анализ результатов

Очевидно, что модель с категориальными признаками показывает несколько более лучшие результаты в предсказании медианной цены домов. Однако нельзя не заметить, что качественный прирост сравнительно невелик. Таким образом, можно заключить:
* либо категориальный признак (близость к океану) в данном случае не сильно релевантен,
* либо не представленно достаточное количество наблюдений (например "островных" наблюдений меньше десятка);
* либо имеет место мультиколлинеарность (возможно, комбинация долготы-широты позволяет заменить категориальный параметр)